In [11]:
import cv2
import sys
from findtools.find_files import (find_files, Match)
import json
import exifread
from facepp import API, File
import os
import pandas as pd
import numpy as np
import time
from pandas.io.json import json_normalize
from pandas import compat
compat.PY3 = True

In [4]:
infile_path = '/Users/dev/Desktop/face/pic/'
outfile_path = '/Users/dev/Desktop/face/cropped/'
path = '/Users/dev/Desktop/face/'

In [9]:
def create_crops_points(found_files, outfile_path):
    api = API('b40d3ea28aad1f2c2aa939dec674abff', '_gYS77zBAL0U60gtv_qebqwRmRWJz4c1')
    faceCascade = cv2.CascadeClassifier(path+'haarcascade_frontalface_default.xml')

    allFaces = []
    allFaces2 = pd.DataFrame()
    facesMeta = pd.DataFrame()
    for idn,found_file in enumerate(list(found_files)[0:5]):
        imgpath = found_file 
        print(imgpath)
        image = cv2.imread(imgpath)
  
        result = api.detection.detect(img = File(found_file),  attribute="pose,age,gender,race")
        print( len(result['face']))
        
        if facesMeta.empty == 'True':
            facesMeta = json_normalize(result['face'])
        else:    
            facesMeta = pd.concat([facesMeta,json_normalize(api.detection.detect(img = File(found_file))['face'])],axis=0)     
                 
        #data = json.loads(str(result))
        #print json.dumps(result)
        
        
        allfacePts = []
        if len(result['face']) > 0:   
            print(str(len(result['face']))+" faces found")    
            
            for idx,face in enumerate(result["face"]):
                
                r2 = api.detection.landmark(face_id = face["face_id"], type = "83p")
                faces2 = json_normalize(r2["result"][0]["landmark"])
                faces2['face_id'] =  face["face_id"]
                
                image = cv2.imread(imgpath)
                
                if allFaces2.empty == 'True':
                    allFaces2 = faces2   
                else:    
                    allFaces2 = pd.concat([allFaces2,faces2],axis=0)
                
                for key,value in r2["result"][0]["landmark"].iteritems():
                    pid=key
                    cx=value["x"]/100*result['img_width']
                    cy=value["y"]/100*result['img_height']
                    cv2.circle(image,(int(cx),int(cy)),1,(0,0,255),1)
                

                #crop image based on key points
                startx=max(0,((face["position"]["center"]["x"]-0.6*face['position']['width'])/100)*result['img_width'])
                starty = max(((face["position"]["center"]["y"]-1.25*face['position']['height'])/100)*result['img_height'],0)
                endx =((face["position"]["center"]["x"]+0.6*face['position']['width'])/100)*result['img_width']
                endy = ((face["position"]["center"]["y"] + 0.8*face['position']['height'])/100)*result['img_height']
             
                
                cropped = image[starty:endy, startx:endx]
                cv2.imshow("cropped", cropped)
                printid=outfile_path + face["face_id"] +".jpeg"
                cv2.imwrite(printid, cropped)
                time.sleep(2)
                cv2.destroyAllWindows()

                #faces2.merge(pd.DataFrame(face["face_id"]))
                r3 = r2["result"][0]["landmark"]

                #print(allFaces2)
                for key,value in r3.iteritems():               
                    facePts = {}
                    facePts["face_id"] = face["face_id"]
                    facePts["position"] = key.encode("ascii","ignore")
                    for k,v in value.iteritems():
                        if k == u'x':
                            facePts["x"] = v
                        if k == u'y':    
                            facePts["y"] = v
                    allfacePts.append(facePts)

            finalFaces = pd.DataFrame(allfacePts)
    return [allFaces2, finalFaces, facesMeta] 

    cv2.destroyAllWindows()

In [10]:
infile_path = '/Users/dev/Desktop/repos/faces/'
infile_path = '/Users/dev/Desktop/repos/faces/test/'
outfile_path = '/Users/dev/Desktop/repos/faces/test/cropped/'
sh_files_pattern = Match(filetype='f', name='*.jp*')
found_files = find_files(path=infile_path, match=sh_files_pattern)
test=create_crops_points(found_files, outfile_path)

http://api.us.faceplusplus.com/
/Users/dev/Desktop/repos/faces/test/facial-expressions.jpg
17
17 faces found
/Users/dev/Desktop/repos/faces/test/smile.jpg
4
4 faces found
/Users/dev/Desktop/repos/faces/test/cropped/0d5753e3e8307728525725ca9a648a88.jpeg
1
1 faces found
/Users/dev/Desktop/repos/faces/test/cropped/0f9ace45a898bb0323953f40335de9d6.jpeg
1
1 faces found
/Users/dev/Desktop/repos/faces/test/cropped/11efb9fc62d97e55f45e9159cf2d6f91.jpeg
1
1 faces found


In [145]:
import math
compat.PY3 = True
pd.set_option('display.max_columns', 200)
type(test)
      
api = API('b40d3ea28aad1f2c2aa939dec674abff', '_gYS77zBAL0U60gtv_qebqwRmRWJz4c1')    
r = api.detection.detect(img = File(found_file),  attribute="pose,age,gender,race")
json_normalize(r['face'])

http://api.us.faceplusplus.com/


,attribute.age.range,attribute.age.value,attribute.gender.confidence,attribute.gender.value,attribute.pose.pitch_angle.value,attribute.pose.roll_angle.value,attribute.pose.yaw_angle.value,attribute.race.confidence,attribute.race.value,face_id,position.center.x,position.center.y,position.eye_left.x,position.eye_left.y,position.eye_right.x,position.eye_right.y,position.height,position.mouth_left.x,position.mouth_left.y,position.mouth_right.x,position.mouth_right.y,position.nose.x,position.nose.y,position.width,tag
0,9,28,99.8487,Female,0.000017,-12.84800,-0.270528,95.2702,White,706ca46ce18f0d11cba1bd72bc2421ed,24.791667,73.000000,17.859042,70.310833,30.054375,68.085667,22.333333,21.801667,80.422500,33.237083,76.837833,24.661458,73.253000,27.916667,
1,5,30,99.9992,Female,-0.000005,-1.47768,-4.793236,99.9901,White,19d85a642da301f90541cf037a375326,74.375000,75.500000,67.283958,71.220333,79.906875,70.959833,21.666667,67.462083,79.835667,80.620833,79.779500,73.177500,76.599167,27.083333,
2,5,29,99.9998,Female,-0.000002,-0.65110,-0.949547,99.9622,White,a2f535d81d553e9ea07987b7354018e3,75.520833,25.416667,68.891667,21.102500,81.265833,20.990000,21.500000,69.919583,31.246333,80.605417,30.837333,74.547083,25.833833,26.875000,
3,9,22,99.9999,Female,0.000002,-1.75504,-1.087993,99.9524,White,05caafabb8dee170d71e7655f8656d1c,26.145833,23.750000,20.364729,19.992333,31.236667,19.725833,19.500000,21.736458,29.450000,29.910417,29.170667,25.532292,23.892500,24.375000,


In [133]:
print(allFaces2.columns[0:100])
print(allFaces2.columns[100:168])

Index(['contour_chin.x', 'contour_chin.y', 'contour_left1.x',
       'contour_left1.y', 'contour_left2.x', 'contour_left2.y',
       'contour_left3.x', 'contour_left3.y', 'contour_left4.x',
       'contour_left4.y', 'contour_left5.x', 'contour_left5.y',
       'contour_left6.x', 'contour_left6.y', 'contour_left7.x',
       'contour_left7.y', 'contour_left8.x', 'contour_left8.y',
       'contour_left9.x', 'contour_left9.y', 'contour_right1.x',
       'contour_right1.y', 'contour_right2.x', 'contour_right2.y',
       'contour_right3.x', 'contour_right3.y', 'contour_right4.x',
       'contour_right4.y', 'contour_right5.x', 'contour_right5.y',
       'contour_right6.x', 'contour_right6.y', 'contour_right7.x',
       'contour_right7.y', 'contour_right8.x', 'contour_right8.y',
       'contour_right9.x', 'contour_right9.y', 'left_eye_bottom.x',
       'left_eye_bottom.y', 'left_eye_center.x', 'left_eye_center.y',
       'left_eye_left_corner.x', 'left_eye_left_corner.y',
       'left_eye_lowe

In [10]:
def distance(df,side1,side2):
    x=((df[side1+'.x'] - df[side2+'.x'])**2 +(df[side1+'.y'] - df[side2+'.y'])**2)**0.5
    return x

def midpoint(df,side1,side2):
    x = (df[side1+'.x'] - df[side2+'.x'])/2.0 + df[side1+'.x'] 
    y = (df[side1+'.y'] - df[side2+'.y'])/2.0 + df[side1+'.y']
    return x , y
    
    
def beauty(df, df2):    
    dfout = pd.DataFrame()  
    dfout['faceId'] = df['face_id']
    dfout['age'] = df2['attribute.age.value']
    dfout['ageRange'] = df2['attribute.age.range']
    dfout['gender'] = df2['attribute.gender.confidence']
    dfout['genderConfidence'] = df2['attribute.gender.value']
    dfout['pitchAngle']  = df2['attribute.pose.pitch_angle.value']
    dfout['rollAngle'] = df2['attribute.pose.roll_angle.value']
    dfout['yawAngle'] = df2['attribute.pose.yaw_angle.value']
    dfout['raceConfidence'] = df2['attribute.race.confidence']
    dfout['race'] = df2['attribute.race.value']
    dfout['innerEyeDist'] = distance(df,'left_eye_right_corner','right_eye_left_corner')
    dfout['pupilDist'] = distance(df,'left_eye_pupil','right_eye_pupil')
    dfout['mouthWidth'] = distance(df,'mouth_right_corner','mouth_left_corner')
    dfout['lowerLipThick'] = distance(df,'mouth_lower_lip_top','mouth_lower_lip_bottom')
    dfout['upperLipThick'] = distance(df,'mouth_upper_lip_top','mouth_upper_lip_bottom')
    dfout['lowerThird'] = distance(df,'mouth_right_corner','mouth_left_corner')
    dfout['noseWidth'] = distance(df, 'nose_right', 'nose_left')
    dfout['eyebrowInnerDist'] = distance(df, left_eyebrow_left_corner )
    df['midpt_pupil.x'] = midpoint(df, 'left_eye_pupil', 'right_eye_pupil')[0]
    df['midpt_pupil.y'] = midpoint(df, 'left_eye_pupil', 'right_eye_pupil')[1]
    dfout['pupilChinDist'] = distance(df, 'midpt_pupil', 'contour_chin')
    dfout['pupilNostrilDist'] = distance(df, 'midpt_pupil', 'nose_contour_lower_middle')
    dfout['pupilCenterLip'] = distance(df, 'midpt_pupil', 'contour_chin')
    dfout['noseTipChinDist'] = distance(df, 'nose_tip', 'contour_chin')
    dfout['nostrilChinDist'] = distance(df, 'nose_contour_lower_middle', 'contour_chin')
    for i in range(1,10):
       df['contourDist'+str(i)] = distance(df, 'contour_left'+str(i), 'contour_right'+str(i))  
    dfout['faceWidth'] = df[['contourDist'+str(i) for i in range(1,10)]].max(axis=1)   
    dfout['ratioPupilDistFaceWidth'] = dfout['pupilDist']/dfout['faceWidth']
    dfout['ratioInnerEyeNoseWidth'] = dfout['innerEyeDist']/dfout['noseWidth']
    dfout['ratioUpperLowerLip'] = dfout['upperLipThick']/dfout['lowerLipThick']
    dfout['ratioPupilDistFaceWidth'] = dfout['pupilDist']/dfout['faceWidth']
    dfout['ratioInnerEyeNoseWidth'] = dfout['innerEyeDist']/dfout['noseWidth']
    dfout['ratioUpperLowerLip'] = dfout['upperLipThick']/dfout['lowerLipThick']
    return dfout
    
allFaces2 = test[0]
metaFaces = test[2]
beauty(allFaces2, metaFaces )

ValueError: cannot reindex from a duplicate axis

In [147]:
test[2]

,attribute.age.range,attribute.age.value,attribute.gender.confidence,attribute.gender.value,attribute.race.confidence,attribute.race.value,attribute.smiling.value,face_id,position.center.x,position.center.y,position.eye_left.x,position.eye_left.y,position.eye_right.x,position.eye_right.y,position.height,position.mouth_left.x,position.mouth_left.y,position.mouth_right.x,position.mouth_right.y,position.nose.x,position.nose.y,position.width,tag
0,5,20,98.8680,Female,99.2276,White,1.049160,7e9e91688641330d27281d277f3d7567,60.630841,63.583333,57.300701,60.828000,64.577570,61.381000,11.166667,57.822196,66.893667,63.015654,67.470667,61.003505,64.221667,15.654206,
1,5,23,98.1934,Female,98.5551,White,19.717400,c5eb88e2fc1e7b21914e62d5ac40f5c0,86.565421,87.583333,82.991589,85.119667,90.265888,85.257333,11.166667,82.982243,91.113167,88.937150,91.345000,87.176636,88.250833,15.654206,
2,5,13,88.4721,Male,99.8536,White,8.966490,e13f5dc10cff7c8087195d10bfe8c6e0,14.135514,87.916667,10.260467,85.875167,18.125093,85.999333,10.833333,10.897500,91.136167,17.098201,91.319500,14.646799,88.726333,15.186916,
3,5,18,80.4434,Female,99.9916,White,6.441200,fc6f4214556158dac737920715f9ae91,37.266355,38.583333,33.542523,36.256000,40.730841,36.261000,10.833333,34.082477,41.709167,40.190187,41.826833,37.513785,39.249000,15.186916,
4,5,19,95.8584,Female,99.8936,White,1.137490,ad5a7ec1559c08054a100853ba390e0a,13.434579,39.083333,10.117547,36.509500,17.245958,36.641500,10.833333,10.484743,42.290833,16.319276,42.454167,13.749836,39.838500,15.186916,
5,5,16,72.0528,Female,99.9815,White,1.785410,2a2ed5891d2a83aa9723a52809fd77ee,60.397196,14.583333,56.805841,12.335100,64.136449,12.372483,10.833333,57.408411,18.375833,63.118925,18.396167,60.231075,15.455533,15.186916,
6,5,18,99.8605,Female,99.9887,White,99.196900,ea4171d6a3bf222e79988087db8b6eb8,37.266355,14.416667,33.584112,11.761950,40.906308,11.930267,10.833333,33.009579,17.016000,40.988084,17.213500,37.574065,15.074500,15.186916,
7,6,14,93.5005,Female,99.6891,White,0.939030,e80676a341c10abdfd6e46590dc37548,37.149533,63.000000,33.645093,60.625500,40.714486,60.729833,10.666667,34.444626,67.229000,39.479907,67.273833,37.477804,63.622000,14.953271,
8,5,8,92.6094,Female,99.3931,White,0.947483,7f9b269f3210cd24fe67b11027f875fa,62.266355,39.083333,60.257477,35.970333,66.892523,37.634167,10.500000,58.121495,42.112667,63.144393,42.924000,62.224533,39.962667,14.719626,
9,5,24,79.5698,Male,99.3592,White,1.982120,bf06480deb55ebfefd1416af5715b571,37.500000,86.583333,33.584813,84.418000,41.076402,84.299833,10.500000,35.353505,91.186833,39.950701,90.830833,37.866589,87.255667,14.719626,


In [42]:
infile_path = '/Users/dev/Desktop/repos/faces/prettyFaces/'
outfile_path='/Users/dev/Desktop/repos/faces/prettyFaces/cropped/'
sh_files_pattern = Match(filetype='f', name='*.jp*')
found_files = find_files(path=infile_path, match=sh_files_pattern)
create_crops_points(found_files, outfile_path)

http://api.us.faceplusplus.com/
/Users/dev/Desktop/repos/faces/prettyFaces/im1.jpeg
1 faces found
(90.49999915000002, 4.249998405, 168.50000035000002, 137.499999594)
/Users/dev/Desktop/repos/faces/prettyFaces/im10.jpeg
1 faces found
(43.900000012, 4.24999806, 117.100000528, 129.300000192)
/Users/dev/Desktop/repos/faces/prettyFaces/im11.jpeg
1 faces found
(73.10000012399999, 0, 179.89999905599998, 167.699999754)
/Users/dev/Desktop/repos/faces/prettyFaces/im12.jpeg
1 faces found
(13.400000424000007, 0, 164.600001336, 218.79999988)
/Users/dev/Desktop/repos/faces/prettyFaces/im13.jpeg
1 faces found
(21.50000025599999, 0, 171.50000102400003, 213.50000152399997)
/Users/dev/Desktop/repos/faces/prettyFaces/im14.jpeg
1 faces found
(25.89999935999999, 0, 147.09999888, 176.30000006400002)
/Users/dev/Desktop/repos/faces/prettyFaces/im15.jpeg
1 faces found
(33.999999768, 0, 190.00000015199998, 228.99999933200004)
/Users/dev/Desktop/repos/faces/prettyFaces/im16.jpeg
1 faces found
(300.377258496, 65.

In [122]:
found_file='/Users/dev/Desktop/repos/faces/pic/smile.jpg'
json_normalize(api.detection.detect(img = File(found_file))['face'])

,attribute.age.range,attribute.age.value,attribute.gender.confidence,attribute.gender.value,attribute.race.confidence,attribute.race.value,attribute.smiling.value,face_id,position.center.x,position.center.y,position.eye_left.x,position.eye_left.y,position.eye_right.x,position.eye_right.y,position.height,position.mouth_left.x,position.mouth_left.y,position.mouth_right.x,position.mouth_right.y,position.nose.x,position.nose.y,position.width,tag
0,9,28,99.8487,Female,95.2702,White,86.43360,204e47b1585df0cfe6a0ed47f31f62a2,24.791667,73.000000,17.859042,70.310833,30.054375,68.085667,22.333333,21.801667,80.422500,33.237083,76.837833,24.661458,73.253000,27.916667,
1,5,30,99.9992,Female,99.9901,White,99.21350,bbe5d2267ddab0c0d73f86024c978539,74.375000,75.500000,67.283958,71.220333,79.906875,70.959833,21.666667,67.462083,79.835667,80.620833,79.779500,73.177500,76.599167,27.083333,
2,5,29,99.9998,Female,99.9622,White,39.78270,b5cbd70f2b8a848a035ab9d86eab05ec,75.520833,25.416667,68.891667,21.102500,81.265833,20.990000,21.500000,69.919583,31.246333,80.605417,30.837333,74.547083,25.833833,26.875000,
3,9,22,99.9999,Female,99.9524,White,1.62544,4bfc09fc6c8981b98ed88e11cf3bfb2f,26.145833,23.750000,20.364729,19.992333,31.236667,19.725833,19.500000,21.736458,29.450000,29.910417,29.170667,25.532292,23.892500,24.375000,


In [60]:
for f in result['face']:
    print(json_normalize(f['attribute']))
    print(f[u'face_id'].encode('ascii','ignore'))
type(result['face'])
print(result)

   age.range  age.value  gender.confidence gender.value  \
0          6         25            99.9999       Female   

   pose.pitch_angle.value  pose.roll_angle.value  pose.yaw_angle.value  \
0                0.000009                16.7227            -22.346358   

   race.confidence race.value  
0          99.2889      White  
3f47752f94fd6c3f951325963a3adb0d
{u'url': None, u'img_id': u'c4e85b83498e199bdf4567d53f2ef438', u'img_width': 640, u'session_id': u'058f01e45ec54f31b0c914deadc490eb', u'face': [{u'attribute': {u'gender': {u'confidence': 99.9999, u'value': u'Female'}, u'age': {u'range': 6, u'value': 25}, u'pose': {u'yaw_angle': {u'value': -22.346358}, u'pitch_angle': {u'value': 9e-06}, u'roll_angle': {u'value': 16.7227}}, u'race': {u'confidence': 99.2889, u'value': u'White'}}, u'face_id': u'3f47752f94fd6c3f951325963a3adb0d', u'tag': u'', u'position': {u'eye_left': {u'y': 36.719167, u'x': 56.471667}, u'center': {u'y': 39.5, u'x': 58.0}, u'width': 9.666667, u'mouth_left': {u'y': 

In [ ]:
# load our image and now use the face detection API to find faces in
# images by uploading an image directly
image = cv2.imread("/Users/dev/Desktop/face/pic/11850366_142441642759382_192245047_n.jpeg")
payload = {"image": open("/Users/dev/Desktop/face/pic/11850366_142441642759382_192245047_n.jpeg", "rb")}
r = requests.post(url, files=payload).json()
print "adrian.jpg: {}".format(r)
 
# loop over the faces and draw them on the image
for (startX, startY, endX, endY) in r["faces"]:
	cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
 
# show the output image
cv2.imshow("adrian.jpg", image)
cv2.waitKey(0)

In [169]:
faces3=pd.DataFrame()
from pandas.io.json import json_normalize
allFaces2 = pd.DataFrame()

In [212]:
faces.columns.values

array(['face_id', 'position', 'x', 'y'], dtype=object)

In [203]:
type(faces2)

pandas.core.frame.DataFrame

In [47]:
#cv2.imread('/Users/dev/Desktop/face/pic/test.jpeg')
cv2.destroyAllWindows()